In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
import pickle

In [2]:
# Load the trained model, scaler pickle file , onehot
model = load_model('model.h5')

# load the encoder and scaler
with open('onehot_encoder_geo.pkl', 'rb') as file:
    label_encoder_geo = pickle.load(file)   

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [18]:
# Example input data
input_data = {
    'CreditScore': 200,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 5,
    'Balance': 10000,
    'NumOfProducts': 1,
    'HasCrCard': 0,
    'IsActiveMember': 0,
    'EstimatedSalary': 20000
}

In [19]:
geo_encoder = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoder, columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

f:\MachineLearning\ANN CLASSIFICATION\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [20]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,200,France,Male,40,5,10000,1,0,0,20000


In [21]:
# Encode categorical variables
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,200,France,1,40,5,10000,1,0,0,20000


In [22]:
# Concatination onehot encoded 
input_df = pd.concat([input_df.drop('Geography', axis=1), geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,200,1,40,5,10000,1,0,0,20000,1.0,0.0,0.0


In [23]:
# Scaling the input data
input_scaled = scaler.transform(input_df)
input_scaled

array([[-4.68707758e+00,  9.13247552e-01,  1.04793593e-01,
        -1.34471949e-03, -1.05836066e+00, -9.16687674e-01,
        -1.54035103e+00, -1.02583358e+00, -1.39843232e+00,
         1.00150113e+00, -5.79467227e-01, -5.76388018e-01]])

In [24]:
# Prediction churn
prediction = model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 87ms/step


array([[0.09048925]], dtype=float32)

In [25]:
prediction_proba = prediction[0][0]

In [26]:
prediction_proba

0.090489246

In [28]:
if prediction_proba > 0.5:
    print("The customer is likely to churn with a probability of {:.2f}%".format(prediction_proba * 100))
else:
    print("The customer is unlikely to churn with a probability of {:.2f}%".format((1 - prediction_proba) * 100))

The customer is unlikely to churn with a probability of 90.95%
